# Testing new AMM


In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import random

# Experiments
from model import run
from model import processing
#from model.plot_utils import *
from model import plot_utils as pu
from model import init_utils

%matplotlib inline

########## AGENT CONFIGURATION ##########
# key -> token name, value -> token amount owned by agent
# note that token name of 'omniABC' is used for omnipool LP shares of token 'ABC'
# omniHDXABC is HDX shares dedicated to pool of token ABC
LP1 = {'omniR1': 500000}
LP2 = {'omniR2': 1500000}
trader = {'HDX': 1000000, 'R1': 1000000, 'R2': 1000000}

# key -> agent_id, value -> agent dict
agent_d = {'Trader': trader, 'LP1': LP1, 'LP2': LP2}
#agent_d = {'Trader': trader, 'LP1': LP1}


########## ACTION CONFIGURATION ##########

action_dict = {
    'sell_hdx_for_r1': {'token_buy': 'R1', 'token_sell': 'HDX', 'amount_sell': 2000, 'action_id': 'Trade', 'agent_id': 'Trader'},
    'sell_r1_for_hdx': {'token_sell': 'R1', 'token_buy': 'HDX', 'amount_sell': 1000, 'action_id': 'Trade', 'agent_id': 'Trader'}
}

# list of (action, number of repititions of action), timesteps = sum of repititions of all actions
trade_count = 5
action_ls = [('trade', trade_count)]

# maps action_id to action dict, with some probability to enable randomness
prob_dict = {
    'trade': {'sell_hdx_for_r1': 0.5,
              'sell_r1_for_hdx': 0.5}
}



########## CFMM INITIALIZATION ##########

# Todo: generalize
initial_values = {
    'token_list': ['R1','R2'],
    'R': [500000,1500000],
    'P': [2,2/3],
    'fee_assets': 0,
    'fee_HDX': 0
}
#initial_values['H'] = [initial_values['Q'] * initial_values['W'][i] for i in range(len(initial_values['token_list']))]
#initial_values['D'] = copy.deepcopy(initial_values['H'])

#amms = [balancer_amm, reweighting_amm]
#amm_types = ['Balancer', 'Reweighting']
#amms = [reweighting_amm]
#amm_types = ['Reweighting']
#labels = amm_types
initial_list = []
config_params = {
    'amm': ['OAMM', 'PAMM'],
    'cfmm_type': "",
    'initial_values': initial_values,
    'agent_d': agent_d,
    'action_ls': action_ls,
    'prob_dict': prob_dict,
    'action_dict': action_dict,
}

config_dict, state = init_utils.get_configuration(config_params)

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:.2f}'.format

run.config(config_dict, state)


In [2]:
print(config_dict)

{'N': 1, 'T': range(0, 5), 'M': {'action_list': [['sell_r1_for_hdx', 'sell_hdx_for_r1', 'sell_hdx_for_r1', 'sell_hdx_for_r1', 'sell_r1_for_hdx']], 'action_dict': [{'sell_hdx_for_r1': {'token_buy': 'R1', 'token_sell': 'HDX', 'amount_sell': 2000, 'action_id': 'Trade', 'agent_id': 'Trader'}, 'sell_r1_for_hdx': {'token_sell': 'R1', 'token_buy': 'HDX', 'amount_sell': 1000, 'action_id': 'Trade', 'agent_id': 'Trader'}}], 'timesteps': [5], 'amm': ['OAMM', 'PAMM']}}


In [3]:
events = run.run()



                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (5, 4, 2, 4)
Execution Method: local_simulations
SimIDs   : [0, 0]
SubsetIDs: [0, 1]
Ns       : [0, 1]
ExpIDs   : [0, 0]
Execution Mode: parallelized
<class 'model.amm.amm_class.OAMM'>
<class 'model.amm.amm_class.OAMM'>
<class 'model.amm.amm_class.OAMM'>
<class 'model.amm.amm_class.OAMM'>
<class 'model.amm.amm_class.OAMM'>
<class 'model.amm.amm_class.PAMM'>
<class 'model.amm.amm_class.PAMM'>
<class 'model.amm.amm_class.PAMM'>
<class 'model.amm.amm_class.PAMM'>
<class 'model.amm.amm_class.PAMM'>
Total execution time: 0.08s


In [4]:
rdf = pd.DataFrame(events)

In [5]:
rdf.head()

,external,AMM,uni_agents,AMM_list,simulation,subset,run,substep,timestep,AMM_parent,AMM_child
0,{},"{'R': [500000, 1500000], 'Q': [1000000, 100000...","{'Trader': {'q': 1000000, 's': [0, 0], 'r': [1...",[],0,0,1,0,0,NaN,NaN
1,{},"{'R': [500000, 1500000], 'Q': [1000000, 100000...","{'Trader': {'q': 1000000, 's': [0, 0], 'r': [1...",OAMM,0,0,1,1,1,<model.amm.amm_class.AMM object at 0x0000016DA...,<model.amm.amm_class.OAMM object at 0x0000016D...
2,{},"{'R': [500000, 1500000], 'Q': [1000000, 100000...","{'Trader': {'q': 1000000, 's': [0, 0], 'r': [1...",OAMM,0,0,1,2,1,<model.amm.amm_class.AMM object at 0x0000016DA...,<model.amm.amm_class.OAMM object at 0x0000016D...
3,{},"{'R': [501000, 1500000], 'Q': [998003.99201596...","{'Trader': {'q': 1001996.0079840319, 's': [0, ...",OAMM,0,0,1,3,1,<model.amm.amm_class.AMM object at 0x0000016DA...,<model.amm.amm_class.OAMM object at 0x0000016D...
4,{},"{'R': [501000, 1500000], 'Q': [998003.99201596...","{'Trader': {'q': 1001996.0079840319, 's': [0, ...",OAMM,0,0,1,4,1,<model.amm.amm_class.AMM object at 0x0000016DA...,<model.amm.amm_class.OAMM object at 0x0000016D...


In [6]:
rdf.tail()

,external,AMM,uni_agents,AMM_list,simulation,subset,run,substep,timestep,AMM_parent,AMM_child
37,{},"{'R': [498005.98800014314, 1500000], 'Q': [100...","{'Trader': {'q': 995996.0079840319, 's': [0, 0...",PAMM,0,1,1,4,4,<model.amm.amm_class.AMM object at 0x0000016DA...,<model.amm.amm_class.PAMM object at 0x0000016D...
38,{},"{'R': [498005.98800014314, 1500000], 'Q': [100...","{'Trader': {'q': 995996.0079840319, 's': [0, 0...",PAMM,0,1,1,1,5,<model.amm.amm_class.AMM object at 0x0000016DA...,<model.amm.amm_class.PAMM object at 0x0000016D...
39,{},"{'R': [498005.98800014314, 1500000], 'Q': [100...","{'Trader': {'q': 995996.0079840319, 's': [0, 0...",PAMM,0,1,1,2,5,<model.amm.amm_class.AMM object at 0x0000016DA...,<model.amm.amm_class.PAMM object at 0x0000016D...
40,{},"{'R': [499005.98800014314, 1500000], 'Q': [100...","{'Trader': {'q': 998008.0158880646, 's': [0, 0...",PAMM,0,1,1,3,5,<model.amm.amm_class.AMM object at 0x0000016DA...,<model.amm.amm_class.PAMM object at 0x0000016D...
41,{},"{'R': [499005.98800014314, 1500000], 'Q': [100...","{'Trader': {'q': 998008.0158880646, 's': [0, 0...",PAMM,0,1,1,4,5,<model.amm.amm_class.AMM object at 0x0000016DA...,<model.amm.amm_class.PAMM object at 0x0000016D...


## Analysis

## Conclusion

